In [1]:
#### To start we will import all of our needed libraries and dive into our datasets.
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.impute import KNNImputer

from py_files.feature_engineering import filter_column_strings, knn_impute, cat_label_pct


In [2]:
df = pd.read_csv('data/post_eda_data.csv')
df.drop(['Unnamed: 0', 'index'], axis=1, inplace=True)

/tmp/ipykernel_123433/1910989435.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/post_eda_data.csv')


In [3]:
df.head()

,index,id,status_group,amount_tsh,funder,gps_height,installer,longitude,latitude,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,pop_height
0,0,69572,functional,6000.0,Roman,1390.0,Roman,34.938093,-9.856322,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,NaN
1,1,8776,functional,NaN,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,NaN
2,2,34310,functional,25.0,Lottery Club,686.0,World vision,37.460664,-3.821329,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,NaN
3,3,67743,non functional,NaN,Unicef,263.0,UNICEF,38.486161,-11.155298,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,NaN
4,4,19728,functional,NaN,Action In A,NaN,Artisan,31.130847,-1.825359,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,NaN


## To Do:

#### String Clean up
   
#### Construction year to age
#### KNNImpute
    - population
    - gps_height
    - amount_tsh¶
    
#### Bin population

#### Mask features based on value count



In [4]:
# String Clean up 

columns_ = [
    'funder',
    'installer',
    'scheme_name',
    'subvillage',
    'ward',
    'basin',
    'region',
    'lga',
    'ward',
    'scheme_management',
    'scheme_name',
    'extraction_type',
    'extraction_type_group',
    'extraction_type_class',
    'management',
    'management_group',
    'payment',
    'payment_type',
    'water_quality',
    'quality_group',
    'quantity',
    'quantity_group',
    'source',
    'source_type',
    'source_class',
    'waterpoint_type',
    'waterpoint_type_group'
]

for col_ in columns_:
    df[col_] = df[col_].str.replace(' ', '') \
        .str.lower() \
        .str.replace('(','') \
        .str.replace(')', '') \
        .str.replace('/', '') \
        .str.replace('_', '')
    
    filter_column_strings(90, df, col_, fuzz.token_sort_ratio, drop=True)

#### Construction Year to Age

In [5]:

df['construction_year'] = 2022 - df['construction_year']
df = df.rename(columns={'construction_year': 'well_age'})

#### Mask low value count features

In [16]:
for col in df.drop('status_group', axis=1).select_dtypes('object').columns.tolist():
    print(col)
    print(df[col].value_counts())
    print()

funder
governmentoftanzania    11299
unknown                  4509
dagida                   3909
hasawa                   2788
nrwssp                   1718
                        ...  
hasneinmuijmbunge           1
mamaku                      1
stp-sustainabletan          1
teamrafiki                  1
velem                       1
Name: funder, Length: 1943, dtype: int64

installer
dwe              21787
unknown           4537
governme          2373
hasawa            1770
rwe               1498
                 ...  
deusndege            1
billyphillips        1
kapelo               1
pnrda                1
majijacharles        1
Name: installer, Length: 1878, dtype: int64

basin
lakevictoria           12871
pangani                11143
rufiji                  9987
internal                9642
laketanganyika          8052
wamiruvu                7577
lakenyasa               6332
ruvumasoutherncoast     5587
lakerukwa               3059
Name: basin, dtype: int64

subvillage
shule   

In [20]:
scheme_mgmt_idx = np.array(np.where(df['scheme_management'] == 'none')).tolist()[0]

df.loc[scheme_mgmt_idx, 'scheme_management'] = 'other'

In [21]:
masked_cols = ['funder', 'installer', 'subvillage', 'ward', 'scheme_name']

for col in masked_cols:
    mask_ = df[col].map(df[col].value_counts()) == 1
    df[col] = df[col].mask(mask_, 'other')

In [22]:
df.to_csv('data/cleaned_data.csv')

In [42]:
df = pd.read_csv('data/cleaned_data.csv')

/tmp/ipykernel_123433/3976309765.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/cleaned_data.csv')


In [43]:
test_df = df[~df.status_group.notna()].copy()
test_df.drop('status_group', axis=1, inplace=True)
df = df[df.status_group.notna()].copy()

In [44]:
df.drop('pop_height', axis=1, inplace=True)
pop_idx = np.array(np.where((df['population'] <= 0) | (df['population'] == 1))).tolist()[0]
df.loc[pop_idx, ['population']] = np.nan

#### KNN Imputation

In [45]:
knn_impute(df, test_df, 'population', [0,1], 66)
knn_impute(df, test_df, 'gps_height', 0, 66)
knn_impute(df, test_df, 'amount_tsh', 0, 5)

(       Unnamed: 0  index     id    status_group   amount_tsh          funder  \
 0               0      0  69572      functional  6000.000000           roman   
 1               1      1   8776      functional   537.928686         grumeti   
 2               2      2  34310      functional    25.000000     lotteryclub   
 3               3      3  67743  non functional   785.979453           unice   
 4               4      4  19728      functional   787.013381           other   
 ...           ...    ...    ...             ...          ...             ...   
 59395       59395  59395  60739      functional    10.000000  germanyrepubli   
 59396       59396  59396  27263      functional  4700.000000     cefa-njombe   
 59397       59397  59397  37057      functional  1200.000000         unknown   
 59398       59398  59398  31282      functional    31.783604           malec   
 59399       59399  59399  26348      functional    20.000000       worldbank   
 
         gps_height    ins

In [7]:
# test_df.to_csv('data/test_set_imputed.txt')
# df.to_csv('data/df_imputed.txt')

df = pd.read_csv('data/df_imputed.txt', index_col=[0])
test_df = pd.read_csv('data/test_set_imputed.txt', index_col=[0])

#### Feature engineer percentage based on functionality

In [8]:
df[['region_code', 'district_code']] = df[['region_code', 'district_code']].astype('object')

In [9]:
columns = df.drop('status_group', axis=1).select_dtypes('object').columns.tolist()
for col_ in columns:
    if col_ != 'status_group':
        df, test_df = cat_label_pct(df, test_df, col_)
        df.drop(col_, axis=1, inplace=True)
        test_df.drop(col_, axis=1, inplace=True)

In [10]:
test_df.fillna(0.0, inplace=True)
df.fillna(0.0, inplace=True)

In [13]:
# Verifying ID's are identical in the test dataset and nan values
og_test_df = pd.read_csv('data/Test_set.csv')
pd.DataFrame(og_test_df['id'] == test_df['id'])['id'].value_counts()

True    14850
Name: id, dtype: int64

In [14]:
test_df.to_csv('data/ml_testing.txt')
df.to_csv('data/ml_training.txt')